# Transport Calculations (attempt #1)
*Akira Di Sandro, 6/22/20*
<br> In this notebook, I will be making some pseudocode for transport calculations.

## 1. Import Packages as usual

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import sectionate

## 2. Open Dataset

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')

* (6/30) Oday is not available for 'uo' and 'vo'
* Also, I don't think I need 'wo' as a variable

* I think for the line above, I will need to find and download the CM4 dataset that I've been using but with a mask variable (look for it using variable) in order to do the transport calculations.
* (update from 6/30) for the mask, Raf said that I can just use NaN's as an indication that there is no data because of land so just using data from non-NaN suffices

In [5]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [6]:
grid = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
grid

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * bnds       (bnds) float64 1.0 2.0
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    forcing_index:           1
    history:                 File was processed by fremetar (GFDL analog of C...
    institution_id:          NOAA-GFDL
    status:                  2019-08-06;created;by nhn2@columbia.edu
    frequency:               mon
    source_id:               GFDL-CM4
    parent_experiment_id:    piControl
    sub_experiment:          none
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    initialization_index:    1
    realization_index:       1
    parent_variant_label:    r1i1p1f1
    variant_info:            N/A
    realm:                   ocean
    branch_time_in_parent:   36500.0
    sub_experiment_id:       none
    data_specs_version:      01.00.27
    institution:             National Oceanic and Atmospheric Administration,...
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    parent_activity_id:      CMIP
    variant_label:           r1i1p1f1
    physics_index:           1
    parent_source_id:        GFDL-CM4
    branch_method:           standard
    nominal_resolution:      25 km
    mip_era:                 CMIP6
    comment:                 <null ref>
    references:              see further_info_url attribute
    parent_time_units:       days since 0001-1-1
    product:                 model-output
    parent_mip_era:          CMIP6
    activity_id:             CMIP
    branch_time_in_child:    0.0
    contact:                 gfdl.climate.model.info@noaa.gov
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    experiment_id:           historical
    grid_label:              gn
    table_id:                Omon
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    source_type:             AOGCM
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    experiment:              historical
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

* The above two lines will also be dependent on the change in line 4.

In [7]:
dog1 = col.search(source_id='GFDL-CM4', variable_id='areacello', grid_label='gn')
dog1.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NOAA-GFDL,GFDL-CM4,piControl,r1i1p1f1,Ofx,areacello,gn,gs://cmip6/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r...,NaN,20180701
1,OMIP,NOAA-GFDL,GFDL-CM4,omip1,r1i1p1f1,Ofx,areacello,gn,gs://cmip6/OMIP/NOAA-GFDL/GFDL-CM4/omip1/r1i1p...,NaN,20180701


In [8]:
dset_dict1 = dog1.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict1.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


['CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn',
 'OMIP.NOAA-GFDL.GFDL-CM4.omip1.Ofx.gn']

In [9]:
grid1 = dset_dict1['CMIP.NOAA-GFDL.GFDL-CM4.piControl.Ofx.gn']

## 3. Open Cluster 

In [10]:
from dask.distributed import Client
from dask_gateway import Gateway


gateway = Gateway()  # connect to Gateway

cluster = gateway.new_cluster()  # create cluster
cluster.scale(10)  # scale cluster

client = Client(cluster)  # connect Client to Cluster

In [11]:
client

Client Scheduler: gateway://traefik-gcp-uscentral1b-prod-dask-gateway.prod:80/prod.6d9407b079554a14a6d1634fb516fa26 Dashboard: https://us-central1-b.gcp.pangeo.io/services/dask-gateway/clusters/prod.6d9407b079554a14a6d1634fb516fa26/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## 4. Actual Calculations

### a. Calculations to get the U/V points

* (6/30) still need to edit these lat and lon to match the observational data

In [12]:
lat1 = -4.49531
lon1 = -208.21236
lat2 = -5.989064
lon2 = -205.234668

In [13]:
isec, jsec, xsec, ysec = sectionate.create_section(grid['lon'], grid['lat'], lon1, lat1, lon2, lat2)

best fit is rounding down


In [23]:
uvpoints = sectionate.transports_C.MOM6_UVpoints_from_section(isec, jsec)
udata = np.empty((6,4))
vdata = np.empty((11,4))
ucount = 0 
vcount = 0

for i in range(len(uvpoints)):
    point = uvpoints[i]
    pttype, i, j = point
    i = int(i)
    j = int(j)
    if pttype == 'U':
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        udata[ucount] = [i, j, lon, lat]  
        ucount += 1
    else:
        lon = grid['lon'].isel(x=i, y=j).values
        lat = grid['lat'].isel(x=i, y=j).values
        vdata[vcount] = [i, j, lon, lat]  
        vcount += 1
# print(f'{point[0]}, {point[1]}, {point[2]}, {lon}, {lat}')

udata should be a 4-by-uvpoints that are u, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the u point.
<br>udata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]
         
vdata should be a 4-by-uvpoints that are v, matrix-like dataset that stores the i and j values and the corresponding longitude and latitude values for the grid point for which we care about the v point.
<br>vdata = [[i values],
<br>         [j values],
<br>         [lon],
<br>         [lat]      ]

### b. Using U/V points to calculate transport
The following are the steps I cam eup with to do the calculation. <br>
* (transport for each cell) T = u * SA (of cell face) * mask
* SA = dy (vertical distance of cell) * depth of cell     (6/30) SA can be replaced with areacello
* Mask = fraction that tells us how much of the cell is in water (mostly 1 but for the edges, it will be some fraction<1)
* Do this calculation for all depths of (i,j) cell and summate


In [25]:
# masked area code from Raf's template
#    this takes care of multiplying the SA by the mask, so all we have to do is use this area 
#    data for transport calculations
area_masked = grid1['areacello'].where(~np.isnan(grid['uo'].isel(time=0)))

In [47]:
vdata

array([[ 368.        ,  484.        , -207.875     ,   -4.74456835],
       [ 369.        ,  484.        , -207.625     ,   -4.74456835],
       [ 370.        ,  483.        , -207.375     ,   -4.9936657 ],
       [ 371.        ,  483.        , -207.125     ,   -4.9936657 ],
       [ 372.        ,  482.        , -206.875     ,   -5.24266911],
       [ 373.        ,  482.        , -206.625     ,   -5.24266911],
       [ 374.        ,  481.        , -206.375     ,   -5.49157286],
       [ 375.        ,  481.        , -206.125     ,   -5.49157286],
       [ 376.        ,  480.        , -205.875     ,   -5.74037266],
       [ 377.        ,  480.        , -205.625     ,   -5.74037266],
       [ 378.        ,  479.        , -205.375     ,   -5.98906374]])

In [48]:
grid['uo'].isel(x=370,y=483,time=0).values

array([[ 2.2833550e-02,  2.1535070e-03, -6.2579155e-02, -9.6854560e-02,
        -8.2416467e-02, -5.0158810e-02, -1.5030318e-02, -2.1484993e-02,
        -5.8522490e-03,  9.4348446e-02,  1.3930008e-01,  1.5044907e-01,
         1.8533663e-01,  2.1557990e-01,  1.7292060e-01,  1.3282315e-01,
         1.0883850e-01,  8.3572417e-02,  6.6959210e-02,  4.3350998e-02,
         1.0826393e-02,  3.9455883e-04,  1.1620312e-04,  9.9628858e-05,
         7.9084661e-05,  5.8539779e-05,  4.1840449e-05,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan]], dtype=float32)

In [50]:
grid['uo'].where(~np.isnan(grid['uo'])).isel(x=370,y=483,time=0).values

array([[ 2.2833550e-02,  2.1535070e-03, -6.2579155e-02, -9.6854560e-02,
        -8.2416467e-02, -5.0158810e-02, -1.5030318e-02, -2.1484993e-02,
        -5.8522490e-03,  9.4348446e-02,  1.3930008e-01,  1.5044907e-01,
         1.8533663e-01,  2.1557990e-01,  1.7292060e-01,  1.3282315e-01,
         1.0883850e-01,  8.3572417e-02,  6.6959210e-02,  4.3350998e-02,
         1.0826393e-02,  3.9455883e-04,  1.1620312e-04,  9.9628858e-05,
         7.9084661e-05,  5.8539779e-05,  4.1840449e-05,            nan,
                   nan,            nan,            nan,            nan,
                   nan,            nan,            nan]], dtype=float32)

In [45]:
total = []  # array that will store transport across each point in udata
for pt in udata:
    i, j, lon, lat = pt
    u = grid['uo'].isel(x=i,y=j,time=0).values
    SAm = area_masked.isel(x=i,y=j).values  # masked surface area of cell
    pt_total = 0
    for row in range(len(SAm)):   # row iterates through the different depths that aren't nan
        if SAm[row] == 'nan':
            pass
        else:
            pt_total += u[row] * SAm[row]
    total = np.append(total, pt_total)
utrans = np.sum(total)

TypeError: invalid indexer array, does not have integer dtype: array(485.)

In [ ]:
total = []  # array that will store transport across each point in vdata
for pt in vdata:
    i, j, lon, lat = pt
    v = grid['vo'].isel(x=i,y=j,time=0).values
    SAm = area_masked.isel(x=i,y=j).values  # masked surface area of cell
    pt_total = 0
    for row in range(len(SAm)):   # row iterates through the different depths that aren't nan
        pt_total += v[row] * SAm[row]
    total = np.append(total, pt_total)
vtrans = np.sum(total)

In [ ]:
transport = utrans + vtrans

### c. Using the above code to make a function that calculates the transport for each time stamp

In [ ]:
def calc_model_transport(ds, t_step, udata, vdata, mask):
    total = []
    for pt in udata:
        i,j,lon,lat = pt
        u = ds['uo'].isel(x=i,y=j,time=t_step).values
        SAm = mask.isel(x=i,y=j).values
        pt_total=0
        for row in range(len(SAm)):
            pt_total += u[row] * SAm[row]
        total = np.append(total, pt_total)
    utrans = np.sum(total)
    total = []
    for pt in vdata:
        i,j,lon,lat = pt
        v = ds['vo'].isel(x=i,y=j,time=t_step).values
        SAm = mask.isel(x=i,y=j).values
        pt_total=0
        for row in range(len(SAm)):
            pt_total += v[row] * SAm[row]
        total = np.append(total, pt_total)
    vtrans = np.sum(total)
    transport = utrans + vtrans
    return transport

## Closing Clusters after use

In [37]:
client.close()
cluster.close()